Sheet 6.1 LLM probing & attribution
========
**Author:** Polina Tsvilodub

* using attention visualization, 
* extracting various representations within a model, 
* understanding how / why to train a classifier for probing   

In this sheet, we will familiarize ourselves with some methods of looking "under the hood" of LLMs. 
**TODO:** Learning goals:

## Attention visualization

One of the core processing mechanisms in the transformer 

In [ ]:
#!pip install bertviz ipywidgets

In [ ]:
# import packages
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model_t5 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [ ]:
from bertviz import model_view
input_ids = tokenizer.encode("Translate to French: The brown dog ran.", return_tensors="pt")
target_ids = tokenizer.encode("Le chien brun a couru.", return_tensors="pt")
# Run model and get the losses of each token
output = model_t5(input_ids=input_ids, labels=target_ids) 
# look at the output of the model
# Retrieve attention from model outputs to see how to access attention scores 
# TODO explain contents of the output
print(output)

# access attention 
attention = output[-1]
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])  # Convert input ids to token strings
model_view(attention, tokens)

**TODO**
* inspect encoder, decoder, cross attentions separately, if possible
* ideally, some example where different attention heads do different things: e.g., Jack's world cappital or IOI task?
* 

> <strong><span style=&ldquo;color:#D83D2B;&rdquo;>Exercise 6.1.1: Interpreting attention scores</span></strong>
>
> Consider the sentence XXX (e.g. world capital). Intuitively, which token do you think will receive high attention scores, from which tokens? Complete the code below and inspect the output. Do the results match your intuiution?

## Gradient tracing

TODO

In [ ]:
# TODO

## Probing

**TODO**

Resources:
* https://github.com/rycolab/probing-via-prompting